In [37]:
from os import environ
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, ArrayType, StructField, StructType, IntegerType
from pyspark.sql.functions import col, to_json, from_json, collect_list

In [26]:
spark = SparkSession.builder.appName("renameJson").getOrCreate()
data_lake = environ['DATA_LAKE']

In [27]:
df = spark.read.csv(data_lake+"csv_rename.csv", header=True)

In [28]:
df.printSchema()

root
 |-- candidate_email: string (nullable = true)
 |-- transactions: string (nullable = true)



In [34]:
st = ArrayType(StructType([StructField("id", IntegerType()), StructField("amount", StringType())]))
st = ArrayType(StructType([StructField("id", StringType()),StructField("amount", StringType())]))

In [38]:
df.withColumn("jsn",from_json(col("transactions"),st).cast("array<struct<id:string,amount:string>>")).\
selectExpr("*","explode(jsn)").\
select("*","col.*").\
selectExpr("candidate_email","to_json(struct(id,amount)) as trans").\
groupBy("candidate_email").\
agg(collect_list("trans").alias("transactions")).\
show(10,False)

+---------------+----------------+
|candidate_email|transactions    |
+---------------+----------------+
|cust2@email.com|[{}, {}]        |
|cust1@email.com|[{}, {}, {}, {}]|
+---------------+----------------+



In [31]:
# spark.stop()